# Validate the trained model and choose a threshold

This jupyter notebook helps user to visualize and validate results form their trained model. Also, it helps usdsr to pick a threshold based on the: 1) density score of the validation dataset (which does not include any images with anomalies), and 2) some known examples of images with anomalies.

In [ ]:
# Import libraries
import json
import os
import random

import keras
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.neighbors import KernelDensity
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from das_anomaly import calculate_percentile, density
from das_anomaly.settings import SETTINGS

## Import parameters and define the generators

In [ ]:
# Import the specified path to the saved trained model from pre-defined config_user.py
results_path = SETTINGS.TRAINED_PATH

# Import the size of the input images
size = SETTINGS.SIZE

# Import the define generators for training, validation and also anomaly data.
batch_size = SETTINGS.BATCH_SIZE
datagen = ImageDataGenerator(rescale=1.0 / 255)

# Path to training PSD plots (seen data)
train_path = SETTINGS.TRAIN_IMAGES_PATH
train_generator = datagen.flow_from_directory(
    train_path, target_size=(size, size), batch_size=batch_size, class_mode="input"
)

# Path to testing PSD plots (unseen data)
test_path = SETTINGS.TEST_IMAGES_PATH
validation_generator = datagen.flow_from_directory(
    test_path, target_size=(size, size), batch_size=batch_size, class_mode="input"
)

# Path to known anomalies
events_path = SETTINGS.ANOMALY_IMAGES_PATH
anomaly_generator = datagen.flow_from_directory(
    events_path, target_size=(size, size), batch_size=batch_size, class_mode="input"
)

In [ ]:
# Load the saved trained model and its history
model_path = os.path.join(results_path, f"model_{size}.h5")
loaded_model = tf.keras.models.load_model(model_path)

# Compile the model with the same trained parameters
loaded_model.compile(
    optimizer="adam", loss="mse", metrics=[keras.metrics.MeanSquaredError()]
)

with open(os.path.join(results_path, f"history_{size}.json")) as json_file:
    history_dict = json.load(json_file)

## Sanity check anomolous and normal, event-free data

### Event-free data

In [ ]:
# Get all batches generated by the datagen and pick a batch for prediction
data_batch_validation = []
batch_num = 0
while (
    batch_num <= validation_generator.batch_index
):  # Gets each generated batch of size batch_size
    data = next(validation_generator)
    data_batch_validation.append(data[0])
    batch_num = +1

predicted = loaded_model.predict(
    data_batch_validation[0]
)  # Predict on the first batch of images

In [ ]:
# Sanity check normal (event-free) data and view an image and corresponding reconstruction
image_number = random.randint(0, predicted.shape[0])
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(data_batch_validation[0][image_number])
plt.title("original data")
plt.subplot(122)
plt.imshow(predicted[image_number])
plt.title("predicted data")
plt.show()

### Anomalous data

In [ ]:
# Get all batches generated by the datagen and pick a batch for prediction
data_batch_anomaly = []
img_num = 0
while (
    img_num <= anomaly_generator.batch_index
):  # Gets each generated batch of size batch_size
    data = next(anomaly_generator)
    data_batch_anomaly.append(data[0])
    img_num = img_num + 1

predicted = loaded_model.predict(
    data_batch_anomaly[0]
)  # Predict on the first batch of images

In [ ]:
# Sanity check anomalous data: plot an image and corresponding reconstructions
image_number = random.randint(0, predicted.shape[0])
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(data_batch_anomaly[0][image_number])
plt.title("original data")
plt.subplot(122)
plt.imshow(predicted[image_number])
plt.title("predicted data")
plt.show()

## Compare the MSE reconstruction error between event-free and anomalous datasets

In [ ]:
# Examine the reconstruction error between our validation data (which are normal, event-free images) and the anomalous images
validation_error = loaded_model.evaluate(validation_generator)

# Evaluate the model using the anomaly data generator
anomaly_error = loaded_model.evaluate(anomaly_generator)

In [ ]:
# Build the encoder network, with trained weights.
# This is used to get the compressed output (latent space) of the input image.
# The compressed output is then used to calculate the KDE and density scores
encoder_model = Sequential()
# Add the convolutional layer without weights
encoder_model.add(
    Conv2D(64, (3, 3), activation="relu", padding="same", input_shape=(size, size, 3))
)
# Set the weights from the corresponding layer of the loaded model
encoder_model.layers[-1].set_weights(loaded_model.layers[0].get_weights())
encoder_model.add(MaxPooling2D((2, 2), padding="same"))
encoder_model.add(Conv2D(32, (3, 3), activation="relu", padding="same"))
encoder_model.layers[-1].set_weights(loaded_model.layers[2].get_weights())
encoder_model.add(MaxPooling2D((2, 2), padding="same"))
encoder_model.add(Conv2D(16, (3, 3), activation="relu", padding="same"))
encoder_model.layers[-1].set_weights(loaded_model.layers[4].get_weights())

encoder_model.add(MaxPooling2D((2, 2), padding="same"))
encoder_model.summary()

## Calculate kernel density estimation (KDE) 

In [ ]:
# Compute latent space representations
encoded_images = encoder_model.predict(train_generator)

# Flatten latent space representations because KDE from sklearn takes 1D vectors as input
encoder_output_shape = encoder_model.output_shape
out_vector_shape = (
    encoder_output_shape[1] * encoder_output_shape[2] * encoder_output_shape[3]
)

encoded_images_vector = [np.reshape(img, (out_vector_shape)) for img in encoded_images]

# Fit a kernel density model to the latent representations
kde = KernelDensity(kernel="gaussian", bandwidth=0.2).fit(encoded_images_vector)

In [ ]:
# Process all_train_batches and all_anomaly_batches as needed
normal_values = density(encoder_model, np.vstack(data_batch_validation), kde)
anomolous_values = density(encoder_model, np.vstack(data_batch_anomaly), kde)

## Plot the histogram and choose a thereshold

In [ ]:
# Convert each observation into “1 / total × 100” so every bar shows % of samples
w_normal = np.ones_like(normal_values) / normal_values.size * 100
w_anom = np.ones_like(anomolous_values) / anomolous_values.size * 100

# Plotting the histogram for density score
plt.hist(
    normal_values, bins=1, weights=w_normal, alpha=0.5, label="data without anomalies"
)
plt.hist(
    anomolous_values, bins=20, weights=w_anom, alpha=0.5, label="data with anomalies"
)

# Adding labels and legend
plt.xlabel("Density score")
plt.ylabel("Percentage of observations (%)")
plt.legend(loc="upper left")

plt.show()

In [ ]:
# Choose a thereshold based on desired percentile and then update the config_user module and use it for anomaly detection (in the detect_anomalies scripts)
percentile = 95
percentile_value = calculate_percentile(anomolous_values, percentile)